# CIFAR-10 Challange - 김영인

1. wide resnet 50-2
2. \+ Data Augmentation(RandomResizedCrop, RandomHorizontalFlip)
3. \+ Data Normalize
5. \+ Optimizer(momentum, scheduler)


In [2]:
import random

import torch
import numpy as np
from matplotlib import pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets

random_seed = 4332
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

device0 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device1 = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

device = device1
print(f"device: {device}") if torch.cuda.is_available() else print("device: cpu")

device: cuda:1


## Hyper parameter Setting 

In [3]:
learning_rate = 0.1
scheduler_step = 30
scheduler_gamma = 0.2
training_epochs = 100
batch_size = 32
momentum = 0.9

## Load & Preprocess Data

In [4]:
transform_train = transforms.Compose(
    [transforms.RandomResizedCrop(224), # data augmentation, 224: image size, ImageNet pretrained model에 맞추기 위해서 224 size로 설정
     transforms.RandomHorizontalFlip(), # data augmentation, 좌우로 대칭
     transforms.ToTensor(), # numpy array를 pytorch tensor로 바꿔주는 역할
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # dataset의 mean, std를 이용해서 -1~1 로 normalize
    ])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Load and Define pretrained model and fine-tuning

In [8]:
print(models.wide_resnet50_2(pretrained=True).fc)

Linear(in_features=2048, out_features=1000, bias=True)


In [9]:
class WideResNet(nn.Module):
    def __init__ (self):
        super(WideResNet, self).__init__()
        self.resnet = models.wide_resnet50_2(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 10)
        
    def forward(self, x):
        x = self.resnet(x)
        return x

## Load Model

In [10]:
net = WideResNet()
net = net.to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net.parameters(), 
                      lr=learning_rate,
                      momentum=momentum)
scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                      step_size = scheduler_step,
                                      gamma = scheduler_gamma)

## Train

In [11]:
net.train()

for epoch in range(training_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = net(inputs) 
        loss =  criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            
        if i == 780:
            class_correct = np.zeros(10)
            class_total = np.zeros(10)

            net.eval()

            with torch.no_grad():
                for data in testloader:
                    images, labels = data
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = net(images)
                    _, predicted = torch.max(outputs, 1)
                    c = (predicted == labels).squeeze()
                    for i in range(4):
                        label = labels[i]
                        class_correct[label] += c[i].item()
                        class_total[label] += 1
            print('==================')
            print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))
            
            net.train()
        
print('Finished Training')

[1,   100] loss: 3.716
[1,   200] loss: 2.324
[1,   300] loss: 2.304
[1,   400] loss: 2.235
[1,   500] loss: 2.170
[1,   600] loss: 2.167
[1,   700] loss: 2.136
Mean Accuracy :  20.41
[1,   800] loss: 2.142
[1,   900] loss: 2.114
[1,  1000] loss: 2.099
[1,  1100] loss: 2.066
[1,  1200] loss: 2.041
[1,  1300] loss: 2.012
[1,  1400] loss: 2.041
[1,  1500] loss: 1.998
[2,   100] loss: 1.998
[2,   200] loss: 1.959
[2,   300] loss: 1.984
[2,   400] loss: 1.952
[2,   500] loss: 1.932
[2,   600] loss: 1.880
[2,   700] loss: 1.918
Mean Accuracy :  27.56
[2,   800] loss: 1.866
[2,   900] loss: 1.895
[2,  1000] loss: 1.857
[2,  1100] loss: 1.872
[2,  1200] loss: 1.861
[2,  1300] loss: 1.874
[2,  1400] loss: 1.851
[2,  1500] loss: 1.818
[3,   100] loss: 1.841
[3,   200] loss: 1.811
[3,   300] loss: 1.788
[3,   400] loss: 1.807
[3,   500] loss: 1.787
[3,   600] loss: 1.775
[3,   700] loss: 1.805
Mean Accuracy :  36.28
[3,   800] loss: 1.764
[3,   900] loss: 1.777
[3,  1000] loss: 1.767
[3,  1100] 

[21,  1200] loss: 0.857
[21,  1300] loss: 0.839
[21,  1400] loss: 0.841
[21,  1500] loss: 0.826
[22,   100] loss: 0.798
[22,   200] loss: 0.800
[22,   300] loss: 0.829
[22,   400] loss: 0.826
[22,   500] loss: 0.862
[22,   600] loss: 0.793
[22,   700] loss: 0.847
Mean Accuracy :  70.58
[22,   800] loss: 0.847
[22,   900] loss: 0.837
[22,  1000] loss: 0.813
[22,  1100] loss: 0.835
[22,  1200] loss: 0.873
[22,  1300] loss: 0.792
[22,  1400] loss: 0.785
[22,  1500] loss: 0.844
[23,   100] loss: 0.828
[23,   200] loss: 0.820
[23,   300] loss: 0.827
[23,   400] loss: 0.818
[23,   500] loss: 0.770
[23,   600] loss: 0.832
[23,   700] loss: 0.795
Mean Accuracy :  69.06
[23,   800] loss: 0.826
[23,   900] loss: 0.804
[23,  1000] loss: 0.798
[23,  1100] loss: 0.800
[23,  1200] loss: 0.809
[23,  1300] loss: 0.804
[23,  1400] loss: 0.813
[23,  1500] loss: 0.811
[24,   100] loss: 0.781
[24,   200] loss: 0.793
[24,   300] loss: 0.798
[24,   400] loss: 0.779
[24,   500] loss: 0.802
[24,   600] loss: 

[42,   400] loss: 0.595
[42,   500] loss: 0.617
[42,   600] loss: 0.554
[42,   700] loss: 0.577
Mean Accuracy :  77.95
[42,   800] loss: 0.570
[42,   900] loss: 0.598
[42,  1000] loss: 0.581
[42,  1100] loss: 0.547
[42,  1200] loss: 0.607
[42,  1300] loss: 0.591
[42,  1400] loss: 0.581
[42,  1500] loss: 0.564
[43,   100] loss: 0.575
[43,   200] loss: 0.561
[43,   300] loss: 0.544
[43,   400] loss: 0.545
[43,   500] loss: 0.581
[43,   600] loss: 0.591
[43,   700] loss: 0.555
Mean Accuracy :  77.23
[43,   800] loss: 0.574
[43,   900] loss: 0.588
[43,  1000] loss: 0.585
[43,  1100] loss: 0.557
[43,  1200] loss: 0.582
[43,  1300] loss: 0.571
[43,  1400] loss: 0.562
[43,  1500] loss: 0.576
[44,   100] loss: 0.551
[44,   200] loss: 0.580
[44,   300] loss: 0.527
[44,   400] loss: 0.561
[44,   500] loss: 0.575
[44,   600] loss: 0.593
[44,   700] loss: 0.584
Mean Accuracy :  78.28
[44,   800] loss: 0.550
[44,   900] loss: 0.544
[44,  1000] loss: 0.575
[44,  1100] loss: 0.568
[44,  1200] loss: 0

[62,   900] loss: 0.447
[62,  1000] loss: 0.474
[62,  1100] loss: 0.499
[62,  1200] loss: 0.465
[62,  1300] loss: 0.469
[62,  1400] loss: 0.464
[62,  1500] loss: 0.478
[63,   100] loss: 0.459
[63,   200] loss: 0.475
[63,   300] loss: 0.457
[63,   400] loss: 0.459
[63,   500] loss: 0.445
[63,   600] loss: 0.478
[63,   700] loss: 0.464
Mean Accuracy :  79.91
[63,   800] loss: 0.494
[63,   900] loss: 0.470
[63,  1000] loss: 0.445
[63,  1100] loss: 0.446
[63,  1200] loss: 0.477
[63,  1300] loss: 0.471
[63,  1400] loss: 0.444
[63,  1500] loss: 0.448
[64,   100] loss: 0.468
[64,   200] loss: 0.442
[64,   300] loss: 0.438
[64,   400] loss: 0.439
[64,   500] loss: 0.443
[64,   600] loss: 0.472
[64,   700] loss: 0.506
Mean Accuracy :  80.01
[64,   800] loss: 0.434
[64,   900] loss: 0.469
[64,  1000] loss: 0.427
[64,  1100] loss: 0.480
[64,  1200] loss: 0.474
[64,  1300] loss: 0.448
[64,  1400] loss: 0.453
[64,  1500] loss: 0.464
[65,   100] loss: 0.449
[65,   200] loss: 0.441
[65,   300] loss: 

[83,   100] loss: 0.365
[83,   200] loss: 0.386
[83,   300] loss: 0.384
[83,   400] loss: 0.395
[83,   500] loss: 0.399
[83,   600] loss: 0.400
[83,   700] loss: 0.395
Mean Accuracy :  80.71
[83,   800] loss: 0.393
[83,   900] loss: 0.381
[83,  1000] loss: 0.391
[83,  1100] loss: 0.396
[83,  1200] loss: 0.357
[83,  1300] loss: 0.373
[83,  1400] loss: 0.398
[83,  1500] loss: 0.407
[84,   100] loss: 0.359
[84,   200] loss: 0.376
[84,   300] loss: 0.375
[84,   400] loss: 0.386
[84,   500] loss: 0.360
[84,   600] loss: 0.400
[84,   700] loss: 0.372
Mean Accuracy :  81.3
[84,   800] loss: 0.384
[84,   900] loss: 0.375
[84,  1000] loss: 0.380
[84,  1100] loss: 0.393
[84,  1200] loss: 0.404
[84,  1300] loss: 0.392
[84,  1400] loss: 0.379
[84,  1500] loss: 0.392
[85,   100] loss: 0.379
[85,   200] loss: 0.401
[85,   300] loss: 0.364
[85,   400] loss: 0.360
[85,   500] loss: 0.394
[85,   600] loss: 0.370
[85,   700] loss: 0.423
Mean Accuracy :  80.86
[85,   800] loss: 0.404
[85,   900] loss: 0.

KeyboardInterrupt: 

## Test

In [12]:
class_correct = np.zeros(10)
class_total = np.zeros(10)

net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))

Accuracy of plane : 81.000000 %
Accuracy of   car : 90.500000 %
Accuracy of  bird : 77.600000 %
Accuracy of   cat : 67.300000 %
Accuracy of  deer : 82.400000 %
Accuracy of   dog : 76.900000 %
Accuracy of  frog : 92.200000 %
Accuracy of horse : 82.700000 %
Accuracy of  ship : 88.000000 %
Accuracy of truck : 85.200000 %
Mean Accuracy :  82.38
